In [ ]:
database_table_definitions = [['Table_Name', 'Column_Name', 'Data_Type', 'Length', 'PK', 'Nullability', 'FK'],
                              ['Car', 'ID', 'VARCHAR', '10', 'Yes', 'No', 'No'],
                              ['Car', 'Body_Style', 'VARCHAR', '20', 'No', 'Yes', 'No'],
                              ['Car', 'Make', 'VARCHAR', '20', 'No', 'Yes', 'No'],
                              ['Car', 'Model', 'VARCHAR', '20', 'No', 'Yes', 'No'],
                              ['Car', 'Price', 'DECIMAL', '9,2', 'No', 'Yes', 'No'],
                              ['Car', 'Trim_Level', 'VARCHAR', '20', 'No', 'Yes', 'No'],
                              ['Car', 'Year', 'DATE', '0', 'No', 'Yes', 'No'],
                              
                              ['Boat', 'ID', 'VARCHAR', '10', 'Yes', 'No', 'No'],
                              ['Boat', 'Flying_Boat', 'BOOLEAN', '0', 'No', 'Yes', 'No'],
                              ['Boat', 'Name', 'VARCHAR', '20', 'No', 'No', 'No'],
                              
                              ['Flying_Boat', 'Boat_ID', 'VARCHAR', '10', 'Yes', 'No', 'Yes'],
                              ['Flying_Boat', 'Plane_ID', 'VARCHAR', '10', 'Yes', 'No', 'Yes'],
                              ['Flying_Boat', 'Information', 'VARCHAR', '100', 'No', 'Yes', 'No'],
                              
                              ['Plane', 'ID', 'VARCHAR', '10', 'Yes', 'No', 'No'],
                              ['Plane', 'Flying_Boat', 'BOOLEAN', '0', 'No', 'Yes', 'No'],
                              ['Plane', 'Name', 'VARCHAR', '20', 'No', 'No', 'No'],
                              
                              ['Vehicle', 'ID', 'VARCHAR', '10', 'Yes', 'No', 'No'],
                              ['Vehicle', 'Kind', 'VARCHAR', '20', 'No', 'No', 'No'],
                              ['Vehicle', 'Details', 'VARCHAR', '10', 'No', 'Yes', 'Yes']
                             ]

In [ ]:
np.savetxt('vehicle_database_sql_table_definitions', database_table_definitions, delimiter = ', ', fmt = '% s')

In [ ]:
db_def = database_table_definitions
current_foreign_keys_list = []
current_primary_keys_list = []
current_table = ''
foreign_keys_dictionary = {}
generated_sql_code = ''
table_definitions_column_indexes = {}
primary_keys_string = ''

for row in range(len(db_def)):
    generated_sql_code_length = len(generated_sql_code)
    #print(generated_sql_code_length)
    
    if(row == 0):
        for column in range(len(db_def[row])):
            table_definitions_column_indexes.update({db_def[row][column]:column})
            #print(table_definitions_column_indexes)
            
    else:  
        if(current_table != db_def[row][table_definitions_column_indexes.get('Table_Name')]):
            
            if(current_table != ''):
                #print(len(current_primary_keys_list))
                
                if(len(current_primary_keys_list) != 0):
                    
                    primary_keys_string = str(current_primary_keys_list)
                    primary_keys_string = primary_keys_string.replace("'", '')
                    
                    generated_sql_code += f'CONSTRAINT {current_table}_pk PRIMARY KEY ({primary_keys_string[1:-1]})'
                
                if(len(current_foreign_keys_list) != 0):
                    foreign_keys_dictionary.update({current_table:current_foreign_keys_list})
                    
                #generated_sql_code = generated_sql_code[:generated_sql_code_length - 6]
                generated_sql_code += '\n);\n\n' 
            
            current_table = db_def[row][table_definitions_column_indexes.get('Table_Name')]
            #print(current_table)
            
            current_primary_keys_list = []
            #print(current_primary_keys_list)
            
            current_foreign_keys_list = []
            #print(current_foreign_keys_list)
            
            primary_keys_string = ''
            
            generated_sql_code += f'CREATE TABLE IF NOT EXISTS {current_table}' + '(' + '\n    '
        
        if(current_table == db_def[row][table_definitions_column_indexes.get('Table_Name')]):
            generated_sql_code += f'{db_def[row][table_definitions_column_indexes.get("Column_Name")]} '
            generated_sql_code += f'{db_def[row][table_definitions_column_indexes.get("Data_Type")]}'
            
            if(db_def[row][table_definitions_column_indexes.get('Data_Type')] != 'DATE' and 
               db_def[row][table_definitions_column_indexes.get('Data_Type')] != 'BOOLEAN'):
                generated_sql_code += f'({db_def[row][table_definitions_column_indexes.get("Length")]})'
            
            if(db_def[row][table_definitions_column_indexes.get('PK')] == 'Yes'):
                current_primary_keys_list.append(db_def[row][table_definitions_column_indexes.get("Column_Name")])
                #print(current_primary_keys_list)
            
            if(db_def[row][table_definitions_column_indexes.get('Nullability')] == 'No'):
                generated_sql_code += ' NOT NULL'
                
            if(db_def[row][table_definitions_column_indexes.get('FK')] == 'Yes'):
                current_foreign_keys_list.append(db_def[row][table_definitions_column_indexes.get("Column_Name")])
            
            generated_sql_code += ',\n    '
            
    if(row == len(db_def) - 1):
        
        if(len(current_primary_keys_list) != 0):    
            primary_keys_string = str(current_primary_keys_list)
            primary_keys_string = primary_keys_string.replace("'", '')
                    
            generated_sql_code += f'CONSTRAINT {current_table}_pk PRIMARY KEY ({primary_keys_string[1:-1]})'
        
        generated_sql_code += '\n);'
        
        if(len(current_foreign_keys_list) != 0):
            foreign_keys_dictionary.update({current_table:current_foreign_keys_list})

In [ ]:
print(foreign_keys_dictionary)

In [ ]:
referenced_foreign_key_table = 'placeholder'

for table in foreign_keys_dictionary:
    
    foreign_key_constraint_count = 0
    
    for foreign_key_column in foreign_keys_dictionary.get(table):
        foreign_key_constraint_count += 1
        generated_sql_code += f'\n\nALTER TABLE {table} ADD CONSTRAINT {table}_fk{foreign_key_constraint_count} FOREIGN KEY ({foreign_key_column}) REFERENCES {referenced_foreign_key_table}({foreign_key_column});'

In [ ]:
print(generated_sql_code)